#### CV Libraries
* torchvision
* torchvision.datasets
* torchvision.models
* torchvision.transforms

In [6]:
import os
import torch
import torchvision
from torch import nn
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

In [2]:
device = 'cuda' if torch.cuda.is_available else 'cpu'

In [8]:
DATA_DIR = 'data/'
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

In [5]:
# get dataset fashion MNIST for testing
train_data = datasets.FashionMNIST(root=DATA_DIR,
                                   train=True,
                                   download=True,
                                   transform=ToTensor(),
                                   target_transform=None)

test_data = datasets.FashionMNIST(root=DATA_DIR,
                                  train=False,
                                  download=True,
                                  transform=ToTensor(),
                                  target_transform=None)

Failed to download (trying next):
<urlopen error [Errno 8] nodename nor servname provided, or not known>



RuntimeError: Error downloading train-images-idx3-ubyte.gz

In [ ]:
len()